In [1]:
# requests 패키지 가져오기
import requests

# 가져올 url 문자열로 입력
url = 'https://s.search.naver.com/p/newssearch/search.naver?'

In [8]:
from urllib.parse import urlparse, parse_qs, parse_qsl, urlencode, urlunparse
from datetime import datetime

params = {
    'field': ['0'],
    'is_dts': ['0'],
    'is_sug_officeid': ['0'],
    'mynews': ['0'],
    'nqx_theme': ['{"theme":{"main":{"name":"corporation_hq"},"sub":[{"name":"stock"}]}}'],
    'nso': ['&nso=so:dd,p:1h,a:all', 'so:dd,p:1h,a:all'],
    'office_category': ['0'],
    'office_section_code': ['0'],
    'office_type': ['0'],
    'photo': ['0'],
    'query': ['삼성전자'],
    'service_area': ['0'],
    'sort': ['1'],
    'spq': ['0'],
    'start': ['11'],
    'where': ['news_tab_api']
}

params['ds'] = ['2024.05.20.00.00']
params['de'] = [datetime.now().strftime("%Y.%m.%d.%H.%M")]
params['sort'] = "1"
params['start'] = "1"

parts = urlparse(url)
parts = parts._replace(query=urlencode(params, doseq=True))
new_url = urlunparse(parts)

In [6]:
from bs4 import BeautifulSoup as bs
import re

headers = {
    # "User-Agent": generate_user_agent(os='win',device_type='desktop'),
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
}

params['ds'] = ['2024.05.20.17.00']
params['de'] = [datetime.now().strftime("%Y.%m.%d.%H.%M")]
params['sort'] = "1"
params['start'] = "1"
parts = urlparse(url)
parts = parts._replace(query=urlencode(params, doseq=True))
new_url = urlunparse(parts)
print(new_url)

total = []

while True:
    response = requests.get(new_url)
    soup = bs(response.content, 'html.parser')
    naver_news_links = [a['href'] for a in soup.find_all('a', string='네이버뉴스') if a['href']]

    for link in naver_news_links:
        tmp = {}
        article_res = requests.get(link[2:-2])
        article_soup = bs(article_res.content, 'html.parser')
        title = article_soup.select_one('#title_area > span').get_text(strip=True)
        tmp['title'] = title
        time = article_soup.select_one('#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div:nth-child(1) > span')['data-date-time']
        tmp['date_news'] = time
        try:
            summary = article_soup.select_one('#dic_area > strong').get_text(strip=True)
        except AttributeError:
            summary = None
        tmp['summary'] = summary
        for tag in article_soup.find_all(class_='end_photo_org'):
            tag.decompose()
        main = article_soup.select_one('#dic_area').get_text(strip=True).replace("\n\n\n", "")
        tmp['content'] = main
        tmp['date_collect'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        tmp['url'] = link[2:-2]

        total.append(tmp)
        if tmp in total:
            break
    params['de'] = [datetime.now().strftime("%Y.%m.%d.%H.%M")]
    params['start'] = str(int(params['start'])+10)
    parts = parts._replace(query=urlencode(params, doseq=True))
    new_url = urlunparse(parts)

https://s.search.naver.com/p/newssearch/search.naver?field=0&is_dts=0&is_sug_officeid=0&mynews=0&nqx_theme=%7B%22theme%22%3A%7B%22main%22%3A%7B%22name%22%3A%22corporation_hq%22%7D%2C%22sub%22%3A%5B%7B%22name%22%3A%22stock%22%7D%5D%7D%7D&nso=%26nso%3Dso%3Add%2Cp%3A1h%2Ca%3Aall&nso=so%3Add%2Cp%3A1h%2Ca%3Aall&office_category=0&office_section_code=0&office_type=0&photo=0&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&service_area=0&sort=1&spq=0&start=1&where=news_tab_api&ds=2024.05.20.17.00&de=2024.05.20.18.17


KeyboardInterrupt: 

In [35]:
import requests
from bs4 import BeautifulSoup as bs
import re
from lxml import html


headers = {
    # "User-Agent": generate_user_agent(os='win',device_type='desktop'),
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
}

new_url = 'https://s.search.naver.com/p/newssearch/search.naver?cluster_rank=38&de=2024.05.27.18.03&ds=2024.05.27.15.03&eid=&field=0&force_original=&is_dts=0&is_sug_officeid=0&mynews=1&news_office_checked=&nlu_query=&nqx_theme=%7B%22theme%22%3A%7B%22main%22%3A%7B%22name%22%3A%22corporation_hq%22%7D%2C%22sub%22%3A%5B%7B%22name%22%3A%22issue%22%7D%2C%7B%22name%22%3A%22stock%22%7D%5D%7D%7D&nso=%26nso%3Dso%3Ar%2Cp%3Aall%2Ca%3Aall&nx_and_query=&nx_search_hlquery=&nx_search_query=&nx_sub_query=&office_category=0&office_section_code=0&office_type=0&pd=9&photo=0&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&query_original=&service_area=0&sort=0&spq=3&start=21&where=news_tab_api&nso=so:r,p:all,a:all&_callback=jQuery1124036467539337756794_1716800613836&_=1716800613839'

total = []

# while True:
response = requests.get(new_url, headers=headers)
# Decode the content properly
decoded_content = response.content.decode('utf-8')

# Parse the HTML content
tree = html.fromstring(decoded_content)
li_elements = tree.xpath('//li')


for index, li in enumerate(li_elements, start=1):
    string = html.tostring(li, pretty_print=True, encoding='unicode')
    soup = bs(string, 'html.parser')
    title_element = soup.find("div").find("div", "class": "\&quot;news_contents\&quot;")
    print(title_element)
    content_element = soup.find(class_="\&quot;api_txt_lines")

    title = title_element.text.strip() if title_element else "No title found"
    content = content_element.text.strip() if content_element else "No content found"

    print(f"Element {index}:")
    print(f"Title: {title}")
    print(f"Content: {content}")
    print("\n")

SyntaxError: invalid syntax (249453642.py, line 29)

In [12]:
# requests 패키지 가져오기
import requests
from urllib.parse import urlparse, urlencode, urlunparse
from datetime import datetime
from bs4 import BeautifulSoup as bs

# 기본 URL 설정
url = 'https://s.search.naver.com/p/newssearch/search.naver?'

# 초기 파라미터 설정
params = {
    'field': ['0'],
    'is_dts': ['0'],
    'is_sug_officeid': ['0'],
    'mynews': ['0'],
    'nqx_theme': ['{"theme":{"main":{"name":"corporation_hq"},"sub":[{"name":"stock"}]}}'],
    'office_category': ['0'],
    'office_section_code': ['0'],
    'office_type': ['0'],
    'photo': ['0'],
    'query': ['삼성전자'],
    'service_area': ['0'],
    'sort': '1',
    'start': '1',
    'where': ['news_tab_api'],
    'ds': ['2024.05.20.00.00'],
    'de': [datetime.now().strftime("%Y.%m.%d.%H.%M")]
}

# URL 생성 함수
def make_url(base_url, params):
    parts = urlparse(base_url)
    parts = parts._replace(query=urlencode(params, doseq=True))
    return urlunparse(parts)



# 수집 데이터 리스트 초기화
total = []

while True:
    new_url = make_url(url, params)
    response = requests.get(new_url, headers=headers)
    soup = bs(response.content, 'html.parser')
    naver_news_links = [a['href'] for a in soup.find_all('a', string='네이버뉴스') if a['href']]

    if not naver_news_links:  # 새로운 데이터가 없을 경우 반복 중단
        break

    for link in naver_news_links:
        tmp = {}
        article_res = requests.get(link[2:-2])
        article_soup = bs(article_res.content, 'html.parser')
        title = article_soup.select_one('#title_area > span').get_text(strip=True)
        tmp['title'] = title
        time = article_soup.select_one('#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div:nth-child(1) > span')['data-date-time']
        tmp['date_news'] = time
        try:
            summary = article_soup.select_one('#dic_area > strong').get_text(strip=True)
        except AttributeError:
            summary = None
        tmp['summary'] = summary
        for tag in article_soup.find_all(class_='end_photo_org'):
            tag.decompose()
        main = article_soup.select_one('#dic_area').get_text(strip=True).replace("\n\n\n", "")
        tmp['content'] = main
        tmp['date_collect'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        tmp['url'] = link[2:-2]

        total.append(tmp)

    params['start'] = str(int(params['start']) + 10)

print(len(total))

106


In [8]:
import requests
from urllib.parse import urlparse, urlencode, urlunparse
from datetime import datetime
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
import argparse

def parse_args():
    parser = argparse.ArgumentParser(description="MongoDB connection parameters")
    parser.add_argument('--host', type=str, required=True, help='MongoDB host address') # 223.130.138.213
    # parser.add_argument('--port', type=int, required=True, help='MongoDB port number') # 30001
    parser.add_argument('--username', type=str, required=True, help='MongoDB username') # root
    parser.add_argument('--password', type=str, default='financial', required=True, help='MongoDB password') # financial
    parser.add_argument('--database', type=str, required=True, help='MongoDB database name')
    parser.add_argument('--query', type=str, required=True, help='검색 쿼리')
    parser.add_argument('--ds', type=str, default=datetime.now().strftime("%Y.%m.%d.%H")+".00", help='시작 일시')
    parser.add_argument('--de', type=str, default=datetime.now().strftime("%Y.%m.%d.%H.%M"), help='종료 일시')
    return parser.parse_args()

def make_url(base_url, params):
    parts = urlparse(base_url)
    parts = parts._replace(query=urlencode(params, doseq=True))
    return urlunparse(parts)

def fetch_news_data(params, headers, url, db):
    total = []
    while True:
        new_url = make_url(url, params)
        response = requests.get(new_url, headers=headers)
        soup = bs(response.content, 'html.parser')
        naver_news_links = [a['href'] for a in soup.find_all('a', string='네이버뉴스') if a['href']]

        if not naver_news_links:
            break

        batch = []
        for link in naver_news_links:
            article_res = requests.get(link[2:-2])
            article_soup = bs(article_res.content, 'html.parser')
            title = article_soup.select_one('#title_area > span').get_text(strip=True)
            
            press_select = article_soup.select_one('#ct > div.media_end_head.go_trans > div.media_end_head_top._LAZY_LOADING_WRAP > a > img.media_end_head_top_logo_img.light_type._LAZY_LOADING._LAZY_LOADING_INIT_HIDE')
            press = press_select['title'] if 'title' in press_select.attrs else 'Title attribute not found'

            time = article_soup.select_one('#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div:nth-child(1) > span')['data-date-time']
            summary = article_soup.select_one('#dic_area > strong').get_text(strip=True) if article_soup.select_one('#dic_area > strong') else None
            content = article_soup.select_one('#dic_area').get_text(strip=True).replace("\n\n\n", "")
            tmp = {
                'date_news': time,
                'query': params['query'],
                'title': title,
                'press': press,
                'summary': summary,
                'content': content,
                'url': link[2:-2],
            }
            batch.append(tmp)

        # 중복 여부 확인 및 저장
        existing_urls = db.news.find({"url": {"$in": [news['url'] for news in batch]}}).distinct("url")
        if existing_urls:
            batch = [news for news in batch if news['url'] not in existing_urls]
            if not batch:
                break

        if batch:
            db.news.insert_many(batch)
            total.extend(batch)

        params['start'] = str(int(params['start']) + 10)

    return total

# def update_mongodb(args, data):
#     client = MongoClient(args.host, args.port, username=args.username, password=args.password)
#     db = client[args.database]
#     db.news.insert_many(data)

def main():
    args = parse_args()
    args.host = 'mongo.stockhelper-mongodb.store'
    args.port = ''
    args.username = 'root'
    args.password = 'financial'
    args.database = 'financial'
    args.query = '삼성전자'
    args.ds = '2024.05.26.00.00'
    args.de = '2024.05.26.16.52'

    params = {
        'filed': '0',
        'is_dts': '0',
        'is_sug_officeid': '0',
        'nso': '&nso=so:dd,p:all,a:all',
        'office_category': '0',
        'office_section_code': '0',
        'service_area': '0',
        'query': args.query,
        'sort': '1',
        'start': '1',
        'where': 'news_tab_api',
        'nso': 'so:dd,p:all,a:all',
        'pd': '0',
        'ds': args.ds,
        'de': args.de
    }
    url = 'https://s.search.naver.com/p/newssearch/search.naver'
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
    }

    # if args.port == '':
    client = MongoClient(args.host, username=args.username, password=args.password)
    # else:
        # client = MongoClient(args.host, args.port, username=args.username, password=args.password)
    db = client[args.database]
    fetch_news_data(params, headers, url, db)

if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/jys/.local/share/jupyter/runtime/kernel-v2-98433BgSOHawR5D8t.json


SystemExit: 2

In [ ]:
"https://s.search.naver.com/p/newssearch/search.naver?cluster_rank=20&de=2024.05.25&ds=2024.05.25&eid=&field=0&force_original=&is_dts=0&is_sug_officeid=0&mynews=1&news_office_checked=&nlu_query=&nqx_theme=%7B%22theme%22%3A%7B%22main%22%3A%7B%22name%22%3A%22corporation_hq%22%7D%2C%22sub%22%3A%5B%7B%22name%22%3A%22stock%22%7D%5D%7D%7D&nso=%26nso%3Dso%3Ar%2Cp%3Afrom20240525to20240525%2Ca%3Aall&nx_and_query=&nx_search_hlquery=&nx_search_query=&nx_sub_query=&office_category=0&office_section_code=0&office_type=0&pd=3&photo=0&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&query_original=&service_area=0&sort=0&spq=0&start=11&where=news_tab_api&nso=so:r,p:from20240525to20240525,a:all&_callback=jQuery11240006142383014574593_1716901788910&_=1716901788912"

In [ ]:
"https://search.naver.com/search.naver?where=news&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2024.05.25&de=2024.05.25&docid=&related=0&mynews=1&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20240525to20240525&is_sug_officeid=0&office_category=0&service_area=0"